In [ ]:
from gensim.models import Word2Vec
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
import gensim.downloader as api
import gensim


import pandas as pd
import numpy as np


In [ ]:
import sys
!{sys.executable} -m pip install wikipedia2vec

In [ ]:
model = wikipedia2vec.Wikipedia2Vec.load("../enwiki_20180420_win10_300d.pkl")

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def get_cosine_similarity(feature_vec_1, feature_vec_2):
    return cosine_similarity(feature_vec_1.reshape(1, -1), feature_vec_2.reshape(1, -1))[0][0]

In [ ]:
img_net = pd.read_csv("../files/ImageNet.csv")
img_net_list = img_net["Class Name"].tolist()

dbpedia = pd.read_csv("../files/dbpedia_classes.csv")
db_list = dbpedia["label"].tolist()

In [ ]:
dataframe = pd.DataFrame(index =db_list ,columns =img_net_list)

In [ ]:
zero_vec = np.zeros(300, dtype=int)
def get_mean_vector_new(word2vec_model, words):
    # remove out-of-vocabulary words
    #words = [word for word in words if word in word2vec_model.key_to_index]
    process_words = []
    for el in words:
        available = False
        try:
            model.get_word_vector(el)
            available = True
        except:
            pass
        if available:
            process_words.append(model.get_word_vector(el))
        else:
            multi = el.split(",")
            #new_val = []
            arr = np.ndarray(shape=(len(multi),300))
            i = 0
            for elem in multi:
                elem = elem.strip()
                mult_words = elem.split(" ")
                elem_av = False
                try:
                    model.get_word_vector(elem)
                    elem_av = True
                except:
                    pass
                if len(mult_words) > 1:
                    mult_vec = np.ndarray(shape=(len(mult_words),300))
                    j = 0
                    for e in mult_words:
                        av = False
                        try:
                            model.get_word_vector(e)
                            av = True
                        except:
                            pass
                        if av:
                            mult_vec[j] = model.get_word_vector(e)
                            j += 1
                        else:
                            mult_vec[j] = zero_vec
                            j += 1
                    arr[i] = np.mean(mult_vec, axis = 0)
                    i += 1
                elif elem_av:
                    #new_val.append(model[elem])
                    arr[i] = model.get_word_vector(elem)
                    i += 1
                else:
                    #new_val.append(zero_vec)
                    arr[i] = zero_vec
                    i += 1
            process_words.append(np.mean(arr, axis=0))#geht das so mit dem np.mean PROBLEM we receive one value but should receive a matrix                

    return process_words

In [ ]:
el_img_vec = get_mean_vector_new(model, img_net_list) # is size 1000 
#el_img_vec[48][57] = 0.0
zero_vec = np.zeros(300, dtype=int)

for i in range(0,len(el_img_vec)):
    for el_db in db_list:
        a = db_list.index(el_db) 
        multi_db = el_db.split(" ")
        if len(multi_db) > 1:
            m_db = np.ndarray(shape=(len(multi_db),300))
            x = 0
            for e in multi_db:
                e_available = False
                try:
                    model.get_word_vector(e)
                    e_available = True 
                except:
                    pass
                
                if e_available:
                    m_db[x] = model.get_word_vector(e)
                    x += 1
                else:
                    m_db[x] = zero_vec
                    x += 1
            dbpedia = np.mean(m_db, axis=0)
            
            sim = get_cosine_similarity(el_img_vec[i],dbpedia)
            dataframe.iat[a,i] = sim            
        else:
            try:
                sim = get_cosine_similarity(el_img_vec[i],model[el_db])
                dataframe.iat[a,i] = sim                
            except:
                dataframe.iat[a,i] = "Not Present"  # this happens if the dbpedia word is not present in the model

In [ ]:
a = dataframe
a = dataframe.replace("Not Present", np.nan)
a =a.replace(0.0, np.nan)

#Now we remove all columns where we have only NaN values
nan = a.dropna(axis=1, how="all")
highest_values_nan = nan.apply(lambda s: s.abs().nlargest(5).index.tolist(), axis=0) #change nlargest for top3 or top5
highest_values_nan

In [ ]:
from pathlib import Path  

filepath = Path('../files/word_embeddings/wiki_2_vec_top5_mean.csv')  

filepath.parent.mkdir(parents=True, exist_ok=True) 

highest_values_nan.to_csv(filepath, index=False)  

Max-Method

In [ ]:
#gives in a list of arrays and returns the mean
def preprocess_array(mult_list, split_criterion:str):
    output_vec = []
    zero_vec = np.zeros(300, dtype=float)
    for el in mult_list.split(split_criterion):
        prep_el = False
        try:
            model.get_word_vector(el)
            prep_el = True 
        except:
            pass
        if prep_el:
            output_vec.append(model.get_word_vector(el))
        else:
            output_vec.append(zero_vec)
    return np.mean(output_vec, axis = 0)

In [ ]:
dataframe_max = pd.DataFrame(index =db_list ,columns =img_net_list)

In [ ]:
#el_img_vec = get_mean_vector_new(model, class_list) # is size 1000 

zero_vec = np.zeros(300, dtype=int)

for el_img in img_net_list:
    for el_db in db_list:
        a = db_list.index(el_db)
        i = img_net_list.index(el_img) 
        multi_db = el_db.split(" ")
        if len(multi_db) > 1:
            m_db = np.ndarray(shape=(len(multi_db),300))
            x = 0
            for e in multi_db:
                e_ava_max = False
                try:
                    model.get_word_vector(e)
                    e_ava_max = True
                except:
                    pass
                if e_ava_max:
                    m_db[x] = model.get_word_vector(e)
                    x += 1
                else:
                    m_db[x] = zero_vec
                    x += 1
            dbpedia = np.mean(m_db, axis=0)
            sim = -1
            for el in el_img.split(","):
                elxx = preprocess_array(model, el, " ")
            
                val = get_cosine_similarity(elxx,dbpedia)
                if val > sim:
                    sim = val
            dataframe_max.iat[a,i] = sim            
        else:
            try:
                sim = -1
                for elem in el_img.split(","):

                    elem = preprocess_array(model, elem, " ")
            
                    val = get_cosine_similarity(elem,model[el_db])
                    if val > sim:
                        sim = val
                #sim = get_cosine_similarity(el_img_vec[i],model[el_db])
                dataframe_max.iat[a,i] = sim                
            except:
                dataframe_max.iat[a,i] = "Not Present" 

In [ ]:
a = dataframe_max
a = dataframe_max.replace("Not Present", np.nan)
a =a.replace(0.0, np.nan)

#Now we remove all columns where we have only NaN values
nan = a.dropna(axis=1, how="all")
highest_values_nan_max = nan.apply(lambda s: s.abs().nlargest(5).index.tolist(), axis=0)
highest_values_nan_max

In [ ]:
from pathlib import Path  

filepath = Path('../files/word_embeddings/wiki_2_vec_top5_max.csv')  

filepath.parent.mkdir(parents=True, exist_ok=True)  

highest_values_nan_max.to_csv(filepath, index=False)  